### Import Libraries

In [1]:
import ccxt
import pandas as pd
import numpy as np
from datetime import date, datetime, timezone, tzinfo
import time, schedule
import matplotlib.pyplot as plt

In [2]:
import dotenv
import os

# Load variables from the .env file
dotenv.load_dotenv(dotenv.find_dotenv("env"))
os.environ["API_KEY"] = os.getenv("API_KEY")
os.environ["API_SECRET"] = os.getenv("API_SECRET")

### Connecting to phemex Exchange

In [3]:
phemex = ccxt.phemex({
    'enableRateLimit': True,
    'apiKey': os.environ["API_KEY"],
    'secret': os.environ["API_SECRET"]
})

In [4]:
print(phemex.fetch_balance())

{'info': {'code': '0', 'msg': '', 'data': [{'currency': 'USD', 'balanceEv': '0', 'lockedTradingBalanceEv': '0', 'lockedWithdrawEv': '0', 'lastUpdateTimeNs': '1700182833697790544', 'walletVid': '0'}, {'currency': 'USDT', 'balanceEv': '0', 'lockedTradingBalanceEv': '0', 'lockedWithdrawEv': '0', 'lastUpdateTimeNs': '1700182833637789465', 'walletVid': '0'}]}, 'USD': {'free': 0.0, 'used': 0.0, 'total': 0.0}, 'USDT': {'free': 0.0, 'used': 0.0, 'total': 0.0}, 'timestamp': 1700182833697, 'datetime': '2023-11-17T01:00:33.697Z', 'free': {'USD': 0.0, 'USDT': 0.0}, 'used': {'USD': 0.0, 'USDT': 0.0}, 'total': {'USD': 0.0, 'USDT': 0.0}}


### Building the Bot

Strategy:

* Determine the trend with sma20_1d
* Buy/Sell to open around the sma20_15m

In [5]:
symbol = 'ETHUSD' #symbol to trade
pos_size = 10 # position size
params = {'timeInForce':'PostOnly'}
target = 25

### Moving Averages

    'timeframes': {                      // empty if the exchange.has['fetchOHLCV'] !== true
        '1m': '1minute',
        '1h': '1hour',
        '1d': '1day',
        '1M': '1month',
        '1y': '1year',
    }

### Determine the Trend

* if bid < sma20_1d then BEARISH
* if bid > sma20_1d then BULLISH

```BUY/SELL TO OPEN AROUND THE SMA20_15m - .1% UNDER AND .3% OVER```

In [6]:
def ask_bid():
    """" ask_bid()[0] == ask"""
    ob = phemex.fetch_order_book(symbol=symbol) # returns the orderbook object wich is a dictionary
    bid = ob['bids'][0][0]
    ask = ob['asks'][0][0]
    return ask, bid

In [7]:
#  FIND DAILY SMA 20
def sma20_1d():

    print('####################### Starting Daily SMA #######################')

    # FETCHING DATA FROM PHEMEX
    timeframe = '1d' # what is the fequency of the bars?
    num_bars = 1000 # how many bars of data will this fetch?
    bars = phemex.fetch_ohlcv(symbol=symbol, timeframe=timeframe, limit=num_bars)

    # BUILDING A DATAFRAME FROM THIS DATA
    df_1d = pd.DataFrame(bars, columns=['timestamp','open','high','low','close','volume'])
    df_1d['timestamp'] = pd.to_datetime(df_1d['timestamp'], unit='ms') # change from milisecond timestamp

    # CALCULATE SMA20d
    df_1d['sma20_1d'] = df_1d['close'].rolling(20).mean()

    # if bid < sma20_1d then BEARISH, if bid > sma20_1d then BULLISH
    bid = ask_bid()[1]
    print(f'##################bid:{bid}####################')
    df_1d.loc[df_1d['sma20_1d']>bid, 'sig'] = 'SELL'
    df_1d.loc[df_1d['sma20_1d']<bid, 'sig'] = 'BUY'
 
    return df_1d

In [8]:
sma20_1d()

####################### Starting Daily SMA #######################


##################bid:1978.61####################


,timestamp,open,high,low,close,volume,sma20_1d,sig
0,2022-06-13,1432.90,1452.70,1161.65,1207.20,130846421.0,NaN,NaN
1,2022-06-14,1207.15,1267.10,1068.90,1206.35,106596975.0,NaN,NaN
2,2022-06-15,1206.35,1250.00,1004.15,1237.05,144729209.0,NaN,NaN
3,2022-06-16,1236.60,1258.40,1050.00,1066.65,186228553.0,NaN,NaN
4,2022-06-17,1066.70,1116.25,1047.90,1084.15,181793108.0,NaN,NaN
...,...,...,...,...,...,...,...,...
517,2023-11-12,2054.82,2069.63,2012.00,2046.78,8527891.0,1878.6815,BUY
518,2023-11-13,2046.78,2120.26,2031.72,2056.03,10480153.0,1892.2605,BUY
519,2023-11-14,2055.63,2066.83,1926.25,1980.08,10451746.0,1901.8930,BUY
520,2023-11-15,1980.51,2064.60,1967.86,2060.47,11528257.0,1914.7335,BUY


Continue with Strategy...

In [9]:
#  FIND 15MINUTE SMA, FIGURE OUT IF BUY OR SELL BASED ON bid vs sma20_1d, if bid < sma20_1d then SELL
def sma20_15m():

    print('####################### Starting 15m SMA #######################')

    # FETCHING DATA FROM PHEMEX
    timeframe = '15m' # what is the fequency of the bars?
    num_bars = 1000 # how many bars of data will this fetch?
    bars = phemex.fetch_ohlcv(symbol=symbol, timeframe=timeframe, limit=num_bars)

    # BUILDING A DATAFRAME FROM THIS DATA
    df_15m = pd.DataFrame(bars, columns=['timestamp','open','high','low','close','volume'])
    df_15m['timestamp'] = pd.to_datetime(df_15m['timestamp'], unit='ms') # change from milisecond timestamp

    # CALCULATE SMA20_15m
    df_15m['sma20_15m'] = df_15m['close'].rolling(20).mean()

    # BUY PRICE 1+2 and SELL PRICE 1+2
    df_15m['bp_1'] = df_15m['sma20_15m'] * 1.001 # sma20_15m 0.1% under and 0.3%over 
    df_15m['bp_2'] = df_15m['sma20_15m'] * 0.997
    df_15m['sp_1'] = df_15m['sma20_15m'] * 0.999
    df_15m['sp_2'] = df_15m['sma20_15m'] * 1.003
    
    return df_15m

In [10]:
sma20_15m()

####################### Starting 15m SMA #######################


,timestamp,open,high,low,close,volume,sma20_15m,bp_1,bp_2,sp_1,sp_2
0,2023-11-06 15:15:00,1912.17,1914.55,1908.75,1912.04,592392.0,NaN,NaN,NaN,NaN,NaN
1,2023-11-06 15:30:00,1912.13,1912.73,1905.00,1908.13,659288.0,NaN,NaN,NaN,NaN,NaN
2,2023-11-06 15:45:00,1908.33,1908.33,1901.67,1902.29,608790.0,NaN,NaN,NaN,NaN,NaN
3,2023-11-06 16:00:00,1902.47,1904.55,1897.55,1899.02,686048.0,NaN,NaN,NaN,NaN,NaN
4,2023-11-06 16:15:00,1898.03,1898.96,1893.15,1897.39,891998.0,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
995,2023-11-17 00:00:00,1962.33,1964.55,1960.79,1961.69,146439.0,1960.0775,1962.037577,1954.197268,1958.117423,1965.957732
996,2023-11-17 00:15:00,1961.68,1970.16,1961.60,1968.20,113930.0,1960.8720,1962.832872,1954.989384,1958.911128,1966.754616
997,2023-11-17 00:30:00,1967.46,1977.99,1966.45,1976.25,157827.0,1961.8070,1963.768807,1955.921579,1959.845193,1967.692421
998,2023-11-17 00:45:00,1976.29,1980.95,1975.38,1979.67,147065.0,1962.4965,1964.458996,1956.609011,1960.534003,1968.383989


### Code the bot

In [11]:
def open_positions():
    params = {'type':'swap', 'code':'USD'}
    phe_bal = phemex.fetch_balance(params=params)
    open_positions = phe_bal['info']['data']['positions']
    open_positions_side = open_positions[0]['side']
    open_positions_size = open_positions[0]['size']

    if open_positions_side == ('Buy'):
        openpos_bool = True
        long = True
    elif open_positions_side == ('Sell'):
        openpos_bool = True
        long = False
    else:
        openpos_bool = False
        long = None
    return open_positions, openpos_bool, open_positions_size, long

In [12]:
# CREATE A KILL_SWITCH
def kill_switch():
    print('starting kill switch')
    openposi = open_positions()[1]
    kill_size = open_positions()[2]
    long = open_positions()[3]

    while openposi == True:
        print('starting kill switch loop till limit fill..')
        temp_df = pd.DataFrame()
        print("just made a temporary df")

        phemex.cancel_all_orders(symbol=symbol)
        openposi = open_positions()[1]
        kill_size = open_positions()[2]
        long = open_positions()[3]

        ask = ask_bid()[0]
        bid = ask_bid()[1]

        if long == False:
            phemex.create_limit_buy_order(symbol=symbol, amount=kill_size, price=bid, params=params)
            print(f"just made a BUY order to close order of {kill_size}|{symbol} at ${bid}")
            print('sleeping for 30 seconds to see if it fills')
            time.sleep(30)
        elif long == True:
            phemex.create_limit_sell_order(symbol=symbol, amount=kill_size, price=ask, params=params)
            print(f"just made a BUY order to close order of {kill_size}|{symbol} at ${ask}")
            print('sleeping for 30 seconds to see if it fills')
            time.sleep(30)
        else:
            print("++++++ Something I didn't expect in kill switch function")

        openposi = open_positions()[1]
    return

In [13]:
kill_switch()

starting kill switch


In [14]:
pos_dict = phemex.fetch_positions(params=params)
pos_dict = pos_dict[0]
pos_dict.keys()

dict_keys(['info', 'id', 'symbol', 'contracts', 'contractSize', 'unrealizedPnl', 'leverage', 'liquidationPrice', 'collateral', 'notional', 'markPrice', 'lastPrice', 'entryPrice', 'timestamp', 'lastUpdateTimestamp', 'initialMargin', 'initialMarginPercentage', 'maintenanceMargin', 'maintenanceMarginPercentage', 'marginRatio', 'datetime', 'marginMode', 'side', 'hedged', 'percentage', 'stopLossPrice', 'takeProfitPrice'])

In [15]:
pos_dict

{'info': {'userID': '4779771',
  'accountID': '47797710002',
  'symbol': 'ALPHAUSD',
  'currency': 'USD',
  'side': 'None',
  'positionStatus': 'Normal',
  'crossMargin': False,
  'leverageEr': '500000000',
  'leverage': '5.00000000',
  'initMarginReqEr': '20000000',
  'initMarginReq': '0.20000000',
  'maintMarginReqEr': '1000000',
  'maintMarginReq': '0.01000000',
  'riskLimitEv': '250000000',
  'riskLimit': '25000.00000000',
  'size': '0',
  'value': '0E-8',
  'valueEv': '0',
  'avgEntryPriceEp': '0',
  'avgEntryPrice': '0E-8',
  'posCostEv': '0',
  'posCost': '0E-8',
  'assignedPosBalanceEv': '0',
  'assignedPosBalance': '0E-8',
  'bankruptCommEv': '0',
  'bankruptComm': '0E-8',
  'bankruptPriceEp': '0',
  'bankruptPrice': '0E-8',
  'positionMarginEv': '0',
  'positionMargin': '0E-8',
  'liquidationPriceEp': '0',
  'liquidationPrice': '0E-8',
  'deleveragePercentileEr': '0',
  'deleveragePercentile': '0E-8',
  'buyValueToCostEr': '20108000',
  'buyValueToCost': '0.20108000',
  'sell

In [16]:
def open_position_for_symbol(symbol):
    pos_dict = phemex.fetch_positions(params=params)
    for i in range(len(pos_dict)):
        try:
            if pos_dict[i]['info']['symbol'] == "ETHUSD":
                return pos_dict[i]
        except Exception as exc:
            print('Error fetching open positions')

In [17]:
# CREATE A PNL CLOSE
def pnl_close():

    current_price = ask_bid()[1]
    # if hit target, close
    print("checking to see if it is time to exit...")
    params = {"type":"swap", "code":"USD"}
    pos_dict = open_position_for_symbol(symbol)
    print(pos_dict)
    side = pos_dict["side"]
    #size = pos_dict["contracts"]
    entry_price = float(pos_dict["entryPrice"])
    leverage = float(pos_dict["leverage"])

    print(f'side:{side} | entry_price:{entry_price} | leverage:{leverage}')

    if side == 'long':
        diff = current_price - entry_price
    else:
        diff = entry_price - current_price
    
    try:
        percentage = round(((diff/entry_price)*leverage), 10)
        print(percentage)
    except:
        percentage = 0
        print(percentage)

    print(f'diff:{diff} | percentage:{percentage}')
    pnl = percentage*100
    #pnl = 26
    
    print(f"this is our pnl {pnl}")

    pnlclose = False
    in_position = False

    if pnl > 0:
        in_position = True
        print("we are in a winning position")
        if pnl > target:
            print(f':) :) :) starting the kill switch becaue we hit our target:{target}')
            pnlclose = True
            kill_switch()
    elif pnl <0:
        print("we are in a loosing position but holding on")
        in_position = True
    else:
        print("we are not in a position")

    return pnlclose, in_position # pnl_close()[0] == pnlclose, pnl_close()[1] == in_position, size 

In [18]:
pnl_close()

checking to see if it is time to exit...
{'info': {'userID': '4779771', 'accountID': '47797710002', 'symbol': 'ETHUSD', 'currency': 'USD', 'side': 'None', 'positionStatus': 'Normal', 'crossMargin': False, 'leverageEr': '500000000', 'leverage': '5.00000000', 'initMarginReqEr': '20000000', 'initMarginReq': '0.20000000', 'maintMarginReqEr': '500000', 'maintMarginReq': '0.00500000', 'riskLimitEv': '10000000000', 'riskLimit': '1000000.00000000', 'size': '0', 'value': '0E-8', 'valueEv': '0', 'avgEntryPriceEp': '0', 'avgEntryPrice': '0E-8', 'posCostEv': '0', 'posCost': '0E-8', 'assignedPosBalanceEv': '0', 'assignedPosBalance': '0E-8', 'bankruptCommEv': '0', 'bankruptComm': '0E-8', 'bankruptPriceEp': '0', 'bankruptPrice': '0E-8', 'positionMarginEv': '0', 'positionMargin': '0E-8', 'liquidationPriceEp': '0', 'liquidationPrice': '0E-8', 'deleveragePercentileEr': '0', 'deleveragePercentile': '0E-8', 'buyValueToCostEr': '20108000', 'buyValueToCost': '0.20108000', 'sellValueToCostEr': '20132000', 's

(False, False)

In [19]:
def bot():
    df_1d = sma20_1d()
    df_15m = sma20_15m()
    ask, bid = ask_bid()
    in_position = pnl_close()[1]

    sig = df_1d.iloc[-1]['sig']
    #sig = 'SELL'
    open_size = pos_size/2

    if in_position == False:
        if sig == "BUY":
            print('making opening order as a BUY')
            bp_1 = df_15m.iloc[-1]['bp_1']
            bp_2 = df_15m.iloc[-1]['bp_2']
            print('bp1:',bp_1)
            print('bp2:',bp_2)

            phemex.cancel_all_orders(symbol=symbol)
            phemex.create_limit_buy_order(symbol=symbol, amount=open_size, price=bp_1, params=params)
            phemex.create_limit_buy_order(symbol=symbol, amount=open_size, price=bp_2, params=params)

            #time.sleep(5)
        else:
            print("making opening order as a SELL")
            sp_1 = df_15m.iloc[-1]['sp_1']
            sp_2 = df_15m.iloc[-1]['sp_2']

            phemex.cancel_all_orders(symbol=symbol)
            phemex.create_limit_sell_order(symbol=symbol, amount=open_size, price=sp_1, params=params)
            phemex.create_limit_sell_order(symbol=symbol, amount=open_size, price=sp_2, params=params)

            #time.sleep(5)
    else:
        print("we are in position already, so not making new orders...")
    return

In [20]:
bot()

####################### Starting Daily SMA #######################
##################bid:1978.79####################
####################### Starting 15m SMA #######################
checking to see if it is time to exit...
{'info': {'userID': '4779771', 'accountID': '47797710002', 'symbol': 'ETHUSD', 'currency': 'USD', 'side': 'None', 'positionStatus': 'Normal', 'crossMargin': False, 'leverageEr': '500000000', 'leverage': '5.00000000', 'initMarginReqEr': '20000000', 'initMarginReq': '0.20000000', 'maintMarginReqEr': '500000', 'maintMarginReq': '0.00500000', 'riskLimitEv': '10000000000', 'riskLimit': '1000000.00000000', 'size': '0', 'value': '0E-8', 'valueEv': '0', 'avgEntryPriceEp': '0', 'avgEntryPrice': '0E-8', 'posCostEv': '0', 'posCost': '0E-8', 'assignedPosBalanceEv': '0', 'assignedPosBalance': '0E-8', 'bankruptCommEv': '0', 'bankruptComm': '0E-8', 'bankruptPriceEp': '0', 'bankruptPrice': '0E-8', 'positionMarginEv': '0', 'positionMargin': '0E-8', 'liquidationPriceEp': '0', 'liquida